In [5]:
!pip install import-ipynb

  Using cached import-ipynb-0.1.3.tar.gz (4.0 kB)
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=d1a0d2624570572b573475be60e8b389e98ce3cb4b4b2680e77247743bc10ac6
  Stored in directory: /Users/bobo/Library/Caches/pip/wheels/06/7e/ad/1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [ ]:
import import_ipynb

In [3]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import csv
import datetime
import time
import os
from selenium import webdriver

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np
import pandas as pd


In [21]:
def set_driver(url):

    ser = Service(DRIVER_PATH)
    options = webdriver.ChromeOptions()  # Set up Chinese
    # options.add_argument('--headless')
    options.add_argument('lang=zh_CN.UTF-8')  # Replacement of head
    options.add_argument(
        'User-Agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"')
    # options.add_argument("--disable-infobars")
    options.add_argument("--headless")
    # PROXY = "11.456.448.110:8080"
    # options.add_argument('--proxy-server=%s' % PROXY)
    options.add_argument("--window-size=1920,1080");
    options.add_argument("--no-sandbox");
    options.add_argument("--disable-extensions");
    options.add_argument("--dns-prefetch-disable");
    options.add_argument("--disable-gpu");
    # options.setPageLoadStrategy(PageLoadStrategy.NORMAL);
    driver = webdriver.Chrome(
        service=ser,
        options=options)

    driver.get(url)
    print("finish setup driver")
    return driver


def init_filename(source, subject):
    # /呱吉/PTT.csv
    # /呱吉/FB.csv
    # ...
    folder = f"/{source}/"
    return subject + folder


def write_file():
    out_filename = ""
    with open(out_filename, 'a') as out_file:
        for line in in_file:

            out_file.write(parsed_line)


def init_csv(file_name):

    with open(file_name, 'w+') as out_file:
        write = csv.writer(out_file)
        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        # PTT, 呱吉吃大便, http://, '2021-11-17', XXXX, XXXXX, 23
        write.writerow(header)


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')


def generate_ytr_name_list(filename="Youtuber名單 - 道歉.csv"):
    ytr_list = []
    with open(filename, 'r') as f:
        rows = csv.reader(f)
        for row in rows:
            if row[1] == "":
                pass
            if row[1] == "頻道(官方帳號)":
                continue
            ytr_name = []
            ytr_name.append(row[1].split("\n")[0])

            ytr_name.extend(row[3].replace("、", "\n").split("\n"))
            ytr_list.append(list(dict.fromkeys(ytr_name)))
    return ytr_list


In [15]:
DRIVER_PATH = '/home/bobo/Desktop/bobo/ML_final/bobo/crawlers/chromedriver'
# DRIVER_PATH = "/Users/bobo/OneDrive - 台灣微軟/0. bobo/Codes/stratechery crawler/chromedriver"
apple_news_url = "https://tw.appledaily.com/search/阿神/"


In [6]:
def scroll_to_button(driver):
    reached_page_end = False
    last_height = driver.execute_script("return document.body.scrollHeight")

    while not reached_page_end:
        driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True
        else:
            last_height = new_height
    print("reach end")
    # driver.quit()


def crawl_page(driver, filename):
    contianer_list = driver.find_element(
        By.CLASS_NAME, "article-container").find_elements(By.CLASS_NAME, "flex-feature")
    
    if contianer_list == []:
        print(contianer_list)
        print("is empty")
        return


    print("len: ", len(contianer_list))

    filename = filename
    with open(filename, 'a+') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for i in contianer_list:
            dic['Source'] = "apple_news"
            dic['Title'] = i.find_element(
                By.CLASS_NAME, 'storycard-headline').text.replace("  ", " ")
            dic['Link'] = i.find_element(
                By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = i.find_element(By.CLASS_NAME, 'timestamp').text
            dic['Summary'] = ""
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)


In [7]:
# tag search
def crawl_apple_news(path, tag_url):
    driver = set_driver(tag_url)

    scroll_to_button(driver)

    time.sleep(10)

    crawl_page(driver, filename=path+"apple_news.csv")

    driver.quit()



In [8]:
ytr_list = generate_ytr_name_list("Youtuber名單 - 道歉.csv")
ytr_list

[['阿神', '芋圓柚子OEUR.studio'],
 ['阿滴英文', '阿滴', '都省瑞'],
 ['Joeman', 'Joeman 九妹'],
 ['木曜4超玩'],
 ['眾量級CROWD', '重量級', '眾量級CROWDxFUN生活'],
 ['谷阿莫'],
 ['黃氏兄弟'],
 ['WACKYBOYS 反骨男孩', 'WACKYBOYS', '反骨男孩', '反骨'],
 ['白癡公主', '戴平雅', 'A減', '痴痴', '王美', '貧乳界女神'],
 ['滴妹', '都冠伶'],
 ['HowFun', 'HowHow'],
 ['聖結石Saint', '聖結石', '曾聖傑'],
 ['放火 Louis', '放火', '李育群'],
 ['愛莉莎莎 Alisasa', '愛莉莎莎'],
 ['理科太太', '陳映彤'],
 ['Hello Catie'],
 ['鍾明軒', '煎熬弟', '國際美人鍾明軒'],
 ["蔡桃貴 蔡阿嘎二伯's Family", '蔡桃貴', '蔡阿嘎', '二伯'],
 ['大蛇丸'],
 ['古娃娃WawaKu', '古娃娃', 'wawawaku'],
 ['米砂Misa', '米砂', 'Misa', '老濕姬', '江佳儒'],
 ['黃大謙', '大謙'],
 ['上班不要看 NSFW', '上班不要看', '上班可以看'],
 ['一隻阿圓 I am CIRCLE', '一隻阿圓', '阿圓'],
 ['Toyz'],
 ['瑀熙 Yuci', '瑀熙', 'Yuci', '大王瑀康熙'],
 ['呱吉', '上班不要看'],
 ['Dr. Ivan 6', 'Dr.Ivan'],
 ['Stand up, Brian! 博恩站起來！', '博恩', '曾博恩', '鋼鋼'],
 ['阿翰po影片', '阿翰'],
 ['龍龍LungLung'],
 ['丹妮婊姐星球', '丹妮婊姐/婊姐'],
 ['孫安佐', 'Edward', 'Sun', '孫華'],
 ['heyitsmindy', 'Mindy'],
 ['VITO維特', '維特'],
 ['綠眼鏡Punk', '綠眼鏡', '徐晢軒'],
 ['NanaQ'],
 ['統神大戲院', '統神', '張嘉航'],
 ['

In [13]:
base_path = "data/"
if not os.path.exists(base_path):
    os.makedirs(base_path)

for ytr_name in ytr_list:
    
    if ytr_name == ['']:
        continue
        
    print(ytr_name)
    folder_path = base_path + ytr_name[0] + "/"
    
    # if folder not exists, create
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)    
    
    # if file exists, continue, else, create
    if os.path.exists(folder_path+"apple_news.csv"):
        pass
    else:
        init_csv(folder_path+"apple_news.csv")
        for name in ytr_name:
            apple_url = "https://tw.appledaily.com/search/"+name+"/"
            try:
                crawl_apple_news(folder_path, apple_url)
            except Exception as e:
                print(e)
                continue    
        
    print("*"*20)
    print()

['阿神', '芋圓柚子OEUR.studio']
finish setup driver
reach end
len:  17
finish setup driver
reach end
[]
is empty
********************

['阿滴英文', '阿滴', '都省瑞']
finish setup driver
reach end
len:  65
finish setup driver
reach end
len:  196
finish setup driver
reach end
len:  2
********************

['Joeman', 'Joeman 九妹']
finish setup driver
reach end
len:  135
finish setup driver
reach end
len:  14
********************

['木曜4超玩']
finish setup driver
reach end
len:  137
********************

['眾量級CROWD', '重量級', '眾量級CROWDxFUN生活']
finish setup driver
reach end
len:  8
finish setup driver
reach end
len:  2018
finish setup driver
reach end
[]
is empty
********************

['谷阿莫']
finish setup driver
reach end
len:  207
********************

['黃氏兄弟']
finish setup driver
reach end
len:  96
********************

['WACKYBOYS 反骨男孩', 'WACKYBOYS', '反骨男孩', '反骨']
finish setup driver
reach end
len:  59
finish setup driver
reach end
len:  66
finish setup driver
reach end
len:  177
finish setup driver
reach en

## crawl web text

In [13]:
from bs4 import BeautifulSoup
import scrapy

In [9]:
def get_filename_from_folder(folder_path, ext=""):
    file_name_list = []
    for file_name in os.listdir(folder_path):
        name = os.path.join(folder_path + "/"+ file_name)
        
        # get file extension
        file_ext = os.path.splitext(file_name)[1]
        # if user give ext and the file extension is not the same, pass
        if ext and file_ext != ext:
            continue
            
        file_name_list.append(name)
        
    return sorted(file_name_list)

In [22]:
driver = set_driver(apple_news_url)

finish setup driver


In [10]:
# print apple news which hasn't finish crawling
for idx, folder_path in enumerate(get_filename_from_folder("./data")):
    for file_path in get_filename_from_folder(folder_path, ".csv"):
                # if not apple news, continue
        if not file_path.split("/")[-1] == "apple_news.csv":
            continue
        
        df = pd.read_csv(file_path)
        
        if df.shape[0] < 2 or df["Text"].notnull().sum() > 2:
            continue
        else:
            print(file_path)


./data/Hello Catie/apple_news.csv
./data/NanaQ/apple_news.csv
./data/孫安佐/apple_news.csv
./data/愛莉莎莎 Alisasa/apple_news.csv
./data/放火 Louis/apple_news.csv
./data/木曜4超玩/apple_news.csv
./data/波特王 Potter King/apple_news.csv
./data/滴妹/apple_news.csv
./data/理科太太/apple_news.csv
./data/瑀熙 Yuci/apple_news.csv
./data/白癡公主/apple_news.csv
./data/眼球中央電視台/apple_news.csv
./data/眾量級CROWD/apple_news.csv
./data/米砂Misa/apple_news.csv
./data/統神大戲院/apple_news.csv
./data/綠眼鏡Punk/apple_news.csv
./data/聖結石Saint/apple_news.csv
./data/蔡桃貴 蔡阿嘎二伯's Family/apple_news.csv
./data/見習網美小吳/apple_news.csv
./data/許藍方博士 Dr. Gracie/apple_news.csv
./data/谷阿莫/apple_news.csv
./data/鍾明軒/apple_news.csv
./data/阿滴英文/apple_news.csv
./data/阿神/apple_news.csv
./data/阿翰po影片/apple_news.csv
./data/黃大謙/apple_news.csv
./data/黃氏兄弟/apple_news.csv
./data/龍龍LungLung/apple_news.csv


In [ ]:
for idx, folder_path in enumerate(get_filename_from_folder("./data")):
    for file_path in get_filename_from_folder(folder_path, ".csv"):
        # if not apple news, continue
        if not file_path.split("/")[-1] == "apple_news.csv":
            continue
        
        print(file_path)
        df = pd.read_csv(file_path)
        
        # if already have data, continue
        if df.shape[0] < 2 or df["Text"].notnull().sum() > 1:
            continue
        else:
            pass

            
        for row, link in enumerate(df.loc[:, "Link"]):
            print(row, link)
            driver.get(link)
            time.sleep(1)
            # soup = BeautifulSoup(driver.page_source, "html.parser")
            
            text = driver.find_element(By.ID, "articleBody").text
            
            df.loc[row,"Text"] = text
            
        df.to_csv(file_path)
        print("*"*20)
        time.sleep(5)

./data/Dr. Ivan 6/apple_news.csv
./data/Gary蓋瑞/apple_news.csv
./data/Hello Catie/apple_news.csv
0 https://tw.appledaily.com/life/20210517/2XQJZB4DXZBLBMLDZSWI2ZRS6M/
1 https://tw.appledaily.com/supplement/20170226/NLXXSCK3DAH67I5SB434GKLQWQ/
********************
./data/HowFun/apple_news.csv
./data/Joeman/apple_news.csv
./data/NanaQ/apple_news.csv
0 https://tw.appledaily.com/life/20211004/WP3M4K6LPJBI5F5IXWMCDRI42E/
1 https://tw.appledaily.com/property/20200915/LL6YAD3QTRHKTIMLYQKCSSZXRU/
********************
./data/Stand up, Brian! 博恩站起來！/apple_news.csv
./data/Toyz/apple_news.csv
./data/VITO維特/apple_news.csv
./data/WACKYBOYS 反骨男孩/apple_news.csv
./data/heyitsmindy/apple_news.csv
./data/一隻阿圓 I am CIRCLE/apple_news.csv
./data/丁特dinter/apple_news.csv
./data/上班不要看 NSFW/apple_news.csv
./data/丹妮婊姐星球/apple_news.csv
./data/古娃娃WawaKu/apple_news.csv
./data/呱吉/apple_news.csv
./data/大蛇丸/apple_news.csv
./data/孫安佐/apple_news.csv
0 https://tw.appledaily.com/local/20211215/5PO5ZX2M2FCZZEBSSHL4IWWCYA/
1